# Init PySpark session

from [this Medium article](https://medium.com/@dipan.saha/pyspark-made-easy-day-2-execute-pyspark-on-google-colabs-f3e57da946a#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjY3MTk2NzgzNTFhNWZhZWRjMmU3MDI3NGJiZWE2MmRhMmE4YzRhMTIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMDc1NTg2ODM5MjQ0NDg3MDEyODciLCJlbWFpbCI6ImNhbWVyb24uY2FsZGVyOTlAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5iZiI6MTcxNjc2MTc5OSwibmFtZSI6IkNhbWVyb24gQy4iLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jSkk0cUNPUHpqdnVjZ2pOSGhvdngyaWQwcU4yc2NjZHAwSmFaa0h1MGdIMEc1YndDWHdnQT1zOTYtYyIsImdpdmVuX25hbWUiOiJDYW1lcm9uIiwiZmFtaWx5X25hbWUiOiJDLiIsImlhdCI6MTcxNjc2MjA5OSwiZXhwIjoxNzE2NzY1Njk5LCJqdGkiOiI1N2IyOGYwOTMyNTZkOWExMTAzNjk1MGFmYzc4Y2RjN2VjNTZjN2QwIn0.c4jbkhrVpjNnAMUv4KQ8q-mTeiDzWfkQMRwiHkgNKldR4LTlv11nPkUC-t0RKptSkfBa85Tt_v9fnDBkiL1PZyMPRBZfu2l4s5Q7zqZ83Jon964GckhSdkMiMfol3wEMTFtgHmzl3yS_9i6mASEKyGgvRSetc-Ke1m_M-YQldeQbuIt_0aAT5w2fZqKvEfGgQNE6BJGFq1i2kTQy4LI956adVloEt0d23t8fNWkfz7jHr-L6yZUsCiLhF1fqIf5W6qu4gNCr1pcs5FoPoxiBb1kNTlJogUvXNvHd-I-uO5mOfVgtao6Qr9tRwIDZ74JSOAvoL984ZUa8cF4dQQJoZg)

In [2]:

!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

# Initialize findspark
import findspark
findspark.init()

# Create a PySpark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,375 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,125 kB]
Fetched 3,733 kB in 2s (1,534 kB/s)
Reading package lists... Do

# Load Data

In [14]:

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import IntegerType


In [15]:

trainFile = "/content/drive/MyDrive/Notebooks/data/trainItem.data"
testFile = "/content/drive/MyDrive/Notebooks/data/testItem.data"


In [12]:

# load training data file into pySpark dataFrame format
training = spark.read.csv(trainFile, header = False)
training = training.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
# assign training dataFrame column data types (dataFrame by default herein assume 'string' type)
training = training.withColumn("userID", training["userID"].cast(IntegerType()))
training = training.withColumn("itemID", training["itemID"].cast(IntegerType()))
training = training.withColumn("rating", training["rating"].cast('float'))

training.show(5)


+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199808|248969|  90.0|
|199808|  2663|  90.0|
|199808| 28341|  90.0|
|199808| 42563|  90.0|
|199808| 59092|  90.0|
+------+------+------+
only showing top 5 rows



In [16]:

# load testing data file into pySpark dataFrame format
testing = spark.read.csv(testFile, header = False)
testing = testing.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")
testing = testing.withColumn("userID", testing["userID"].cast(IntegerType()))
testing = testing.withColumn("itemID", testing["itemID"].cast(IntegerType()))
testing = testing.withColumn("rating", testing["rating"].cast('float'))

testing.show(5)


+------+------+------+
|userID|itemID|rating|
+------+------+------+
|199810|208019|   0.0|
|199810| 74139|   0.0|
|199810|  9903|   0.0|
|199810|242681|   0.0|
|199810| 18515|   0.0|
+------+------+------+
only showing top 5 rows



In [18]:

# Create ALS model
als = ALS(
    maxIter=5,
    rank = 5,
    regParam=0.01,
    userCol="userID",
    itemCol="itemID",
    ratingCol="rating",
    nonnegative = True,
    implicitPrefs = False,
    coldStartStrategy="drop"
)
# fit the ALS model using the training set
model = als.fit(training)


In [19]:

# predict ratings with testing
predictions = model.transform(testing)
predictions.show(5)


+------+------+------+----------+
|userID|itemID|rating|prediction|
+------+------+------+----------+
|230073|   463|   0.0| 123.03484|
|230962|   471|   0.0|  86.49538|
|218845|  1088|   0.0| 153.73868|
|209697|  1088|   0.0|  46.83917|
|224445|  2142|   0.0| 30.431276|
+------+------+------+----------+
only showing top 5 rows



In [20]:


# save dataframe into a folder 'predictions' with a single file (coalesce(1))
# note: cannot assign the filename
predictions.coalesce(1).write.csv("predictions")

# save predictions result to a file
predictions.toPandas().to_csv('myprediction.csv')
